In [32]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [33]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [34]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # READ; as per lines

    dx = open('university_towns.txt','r')
    df = dx.readlines()



    lst = []

    # WHY seperate string??

    # state = ''
    # region = ''
    #print (df)
    for val in df: 
        
        # STATE has EDIT
        if ("[edit]" in val):
            x = val.split('[edit]')
            state = x[0] 

        else:
        # if not EDIT: consider REGION
            a = val.split(' (')
            region = a[0]
            lst.append([state,region])


    dff = pd.DataFrame(lst, columns = ['State','RegionName'])
    

    return dff


In [37]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows = 5)
    # col: RENAME
    df.rename(columns = {'Unnamed: 0': 0,'Unnamed: 4' : 'Quarter', 'GDP in billions of chained 2009 dollars.1':'GDP 2009'},inplace = True)

    #col: DROP
    df.drop(['Unnamed: 3','Unnamed: 7','GDP in billions of current dollars.1'], axis = 1, inplace =True)

    #req :QUARTER >= 2000 
    df = df[df['Quarter'] >= '2000q1']

    #col : DROP
    df = df.iloc[:,3:].reset_index()

    #reset: INDEX
    gdp = df.drop(['index'], axis = 1)
    gdp
    
    for i in range(len(gdp)):
        
        #len(gdp) = 65 
        # if i =65 then condition not fullfilled.
        if i != 65:
            if gdp.loc[i]['GDP 2009']>gdp.loc[i+1]['GDP 2009'] > gdp.loc[i+2]['GDP 2009'] > gdp.loc[i+3]['GDP 2009'] < gdp.loc[i+4]['GDP 2009']:
                start=gdp.loc[i]['Quarter']
                
    return start


'2008q3'

In [39]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''


    df = pd.read_excel('gdplev.xls', skiprows = 5)
    # col: RENAME
    df.rename(columns = {'Unnamed: 0': 0,'Unnamed: 4' : 'Quarter', 'GDP in billions of chained 2009 dollars.1':'GDP 2009'},inplace = True)

    #col: DROP
    df.drop(['Unnamed: 3','Unnamed: 7','GDP in billions of current dollars.1'], axis = 1, inplace =True)

    #req :QUARTER >= 2000 
    df = df[df['Quarter'] >= '2000q1']

    #col : DROP
    df = df.iloc[:,3:].reset_index()

    #reset: INDEX
    gdp = df.drop(['index'], axis = 1)
    gdp

    df_2008 = gdp[gdp['Quarter'] >= '2008q3']
    df_2008.reset_index(inplace = True)
    #print(df_2008)

    #RECESSION END; when consecutive GDP is +ve.
    for i in range(len(df_2008)):
        if df_2008.loc[i+1]['GDP 2009'] - df_2008.loc[i]['GDP 2009']  > 0:
            end = df_2008.loc[i+2]['Quarter']
            break
    return end

'2009q4'

In [40]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows = 5)
    # col: RENAME
    df.rename(columns = {'Unnamed: 0': 0,'Unnamed: 4' : 'Quarter', 'GDP in billions of chained 2009 dollars.1':'GDP 2009'},inplace = True)

    #col: DROP
    df.drop(['Unnamed: 3','Unnamed: 7','GDP in billions of current dollars.1'], axis = 1, inplace =True)

    #req :QUARTER >= 2000 
    df = df[df['Quarter'] >= '2000q1']

    #col : DROP
    df = df.iloc[:,3:].reset_index()

    #reset: INDEX
    gdp = df.drop(['index'], axis = 1)
    gdp
    for i in range(len(df_2008)):
        if df_2008.loc[i]['GDP 2009'] - df_2008.loc[i+1]['GDP 2009'] < 0:
            bottom = df_2008.loc[i]['Quarter']
            break
    return bottom

In [41]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')

    # set index
    df.set_index(['State','RegionName'], inplace = True)
    df.head()

    #collect columns from 2001-01 to 2016-01
    df = df.loc[:,'2000-01':]
    df.head()

    # MAP; state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    df.reset_index(inplace =True)
    df.replace({'State':states}, inplace = True)
    df.set_index(['State','RegionName'], inplace = True)

    # CONVERT; columns to datetime to resample into Quarters and take the mean by axis = 1

    df.columns = pd.to_datetime(df.columns)
    df = df.apply(pd.to_numeric)
    #Q - QUARTER
    df = df.resample('Q', axis = 1).mean()

    # RENAME; columns to q1,q2,q3,q4
    year = 2000
    quarter = 1
    for column in df:
        df.rename(columns={column:str(year)+'q'+str(quarter)}, inplace=True)
        if quarter < 4:
            quarter += 1
        else:
            year += 1
            quarter = 1
    return df

In [62]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()

    #price_ratio=quarter_before_recession/recession_bottom
    #start/end
    start = get_recession_start()
    bottom = get_recession_bottom()
    end =  get_recession_end()
    df = df[['2008q2',bottom]]
    df['price_ratio'] = df['2008q2']/df[bottom]
    df.reset_index(inplace =True)
    df.head()
    dx = get_list_of_university_towns()

    # housing_Town in Univ_Town (town = regionName) 

    univ_town = df[df['RegionName'].isin(dx['RegionName'])]
    univ_mean = univ_town['price_ratio'].mean()
    # housing_Town not in Univ_Town (town = regionName) 

    non_univ_town = df[-df['RegionName'].isin(dx['RegionName'])]
    non_mean = non_univ_town['price_ratio'].mean()


    result = ttest_ind(univ_town['price_ratio'], non_univ_town['price_ratio'], nan_policy= 'omit')

    p = result[1]
    if univ_mean < non_mean:
        better = 'university town'
    else:
        better = 'non-university town'

    if result[1] < 0.01:
        different = True
    else:
        different = False

    return (different, p, better)